In [3]:
import pandas as pd
from google.colab import files
files=files.upload()
df = pd.read_csv('Emotion_classify_Data.csv')
df


Saving Emotion_classify_Data.csv to Emotion_classify_Data.csv


,Comment,Emotion
0,i seriously hate one subject to death but now ...,fear
1,im so full of life i feel appalled,anger
2,i sit here to write i start to dig out my feel...,fear
3,ive been really angry with r and i feel like a...,joy
4,i feel suspicious if there is no one outside l...,fear
...,...,...
5932,i begun to feel distressed for you,fear
5933,i left feeling annoyed and angry thinking that...,anger
5934,i were to ever get married i d have everything...,joy
5935,i feel reluctant in applying there because i w...,fear


In [6]:
df.Emotion.value_counts()

,count
Emotion,
anger,2000
joy,2000
fear,1937


In [9]:
min_columns = 1937

df_anger = df[df.Emotion == 'anger'].sample(min_columns, random_state=2022)
df_joy = df[df.Emotion == 'joy'].sample(min_columns, random_state=2022)
df_fear = df[df.Emotion == 'fear'].sample(min_columns, random_state=2022)

In [11]:
df = pd.concat([df_anger, df_joy, df_fear], axis=0)
df

,Comment,Emotion
2973,i constantly feel lied to and wronged by them ...,anger
3785,i feel hated i feel like i dont belong and mor...,anger
1860,i could ingrain in my mind all my feelings all...,anger
5478,i dwell on this matter the more i feel infuria...,anger
4939,i just feel too stubborn to give up on a dream,anger
...,...,...
5268,i suppose to feel terrified,fear
1949,i notice a lump or feel pain in any part of my...,fear
541,i feel doubtful even when i am struggling a bi...,fear
3824,i don t want to go home to toronto and feel li...,fear


In [13]:
df.Emotion.value_counts()

,count
Emotion,
anger,1937
joy,1937
fear,1937


In [16]:
df['Emotion_num'] = df['Emotion'].map({
    'anger': 0,
    'joy': 1,
    'fear':2
})
df

,Comment,Emotion,Emotion_num
2973,i constantly feel lied to and wronged by them ...,anger,0
3785,i feel hated i feel like i dont belong and mor...,anger,0
1860,i could ingrain in my mind all my feelings all...,anger,0
5478,i dwell on this matter the more i feel infuria...,anger,0
4939,i just feel too stubborn to give up on a dream,anger,0
...,...,...,...
5268,i suppose to feel terrified,fear,2
1949,i notice a lump or feel pain in any part of my...,fear,2
541,i feel doubtful even when i am struggling a bi...,fear,2
3824,i don t want to go home to toronto and feel li...,fear,2


In [17]:
import spacy

nlp = spacy.load("en_core_web_sm")

def preprocess(text):
  doc = nlp(text)
  filtered_tokens = []

  for token in doc:
    if token.is_stop or token.is_punct:
      continue
    filtered_tokens.append(token.lemma_)

  return(' ').join(filtered_tokens)

In [18]:
# Modelling without Pre-processing Text data

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.Comment, df.Emotion_num, test_size=0.2, random_state=2022, stratify=df.Emotion_num)

In [29]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer

clf = Pipeline([
     ('vectorizer_tfidf', TfidfVectorizer()),
     ('KNN', KNeighborsClassifier())
])

clf.fit(X_train, y_train)

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('KNN', KNeighborsClassifier())])

In [30]:
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.68      0.80      0.74       388
           1       0.74      0.69      0.72       388
           2       0.77      0.68      0.72       387

    accuracy                           0.73      1163
   macro avg       0.73      0.73      0.73      1163
weighted avg       0.73      0.73      0.73      1163



In [33]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

# Create a pipeline object
clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()),
    ('multinomial_nb', MultinomialNB())
])


In [34]:
clf.fit(X_train, y_train)

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('multinomial_nb', MultinomialNB())])

In [36]:
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.90      0.90      0.90       388
           1       0.92      0.87      0.89       388
           2       0.88      0.93      0.90       387

    accuracy                           0.90      1163
   macro avg       0.90      0.90      0.90      1163
weighted avg       0.90      0.90      0.90      1163



In [37]:
df['preprocess_Comment'] = df['Comment'].apply(preprocess)
df

,Comment,Emotion,Emotion_num,preprocess_Comment
2973,i constantly feel lied to and wronged by them ...,anger,0,constantly feel lied wrong love people death
3785,i feel hated i feel like i dont belong and mor...,anger,0,feel hated feel like not belong feel want die
1860,i could ingrain in my mind all my feelings all...,anger,0,ingrain mind feeling experience read hate happ...
5478,i dwell on this matter the more i feel infuria...,anger,0,dwell matter feel infuriate m lowly thought
4939,i just feel too stubborn to give up on a dream,anger,0,feel stubborn dream
...,...,...,...,...
5268,i suppose to feel terrified,fear,2,suppose feel terrified
1949,i notice a lump or feel pain in any part of my...,fear,2,notice lump feel pain body fearful scared
541,i feel doubtful even when i am struggling a bi...,fear,2,feel doubtful struggle bit faith time dark fee...
3824,i don t want to go home to toronto and feel li...,fear,2,don t want home toronto feel like torture arti...


In [38]:
X_train, X_test, y_train, y_test = train_test_split(df.preprocess_Comment, df.Emotion_num, test_size=0.2, random_state=2022, stratify=df.Emotion_num)

In [39]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

# Create a pipeline object
clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()),
    ('multinomial_nb', MultinomialNB())
])

In [40]:
clf.fit(X_test, y_test)

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('multinomial_nb', MultinomialNB())])

In [41]:
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.99      0.99       388
           1       0.99      0.99      0.99       388
           2       0.98      0.99      0.99       387

    accuracy                           0.99      1163
   macro avg       0.99      0.99      0.99      1163
weighted avg       0.99      0.99      0.99      1163

